In [4]:
import shutil
source = "database/hotel_reservation_sample.db"
db_file = "database/working_db.db"
shutil.copyfile(source, db_file) #Copy original DB first to always start fresh.

'database/working_db.db'

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

conn = sqlite3.connect("database/hotel_reservation_sample.db")

query = """
SELECT rt.description AS room_type, COUNT(*) AS total_bookings
FROM Booking b
JOIN Room r ON b.room_id = r.room_id
JOIN Room_Type rt ON r.type_id = rt.type_id
WHERE b.is_cancelled = 0
GROUP BY rt.description
"""

df = pd.read_sql_query(query, conn)
conn.close()

# Visualisierung
df.plot(kind="bar", x="room_type", y="total_bookings", legend=False, title="Beliebtheit der Zimmertypen")
plt.ylabel("Buchungen")
plt.xlabel("Zimmertyp")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

conn = sqlite3.connect("database/hotel_reservation_sample.db")

query = """
SELECT strftime('%Y-%m', check_in_date) AS month, SUM(total_amount) AS revenue
FROM Booking
WHERE is_cancelled = 0
GROUP BY month
ORDER BY month
"""
df = pd.read_sql_query(query, conn)
conn.close()

df.plot(kind="line", x="month", y="revenue", marker='o', title="Revenue Trend")
plt.ylabel("CHF")
plt.xlabel("Monat")
plt.grid(True)
plt.tight_layout()
plt.show()

conn = sqlite3.connect("database/hotel_reservation_sample.db")

query = """
SELECT AVG(alter) AS avg_age FROM Guest
"""
df = pd.read_sql_query(query, conn)
conn.close()

print("Durchschnittsalter der Gäste:", round(df['avg_age'][0], 1))
